In [1]:
import torch, numpy as np
from utils import unpack_sym

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ms_file = 'data/feature_engineering_data.h5'


In [2]:
import sys
sys.path.append('./feature_engineering/')
import feature_extraction as extract
from utils import MicrostructureImageDataset

MS_train = MicrostructureImageDataset(file_path=ms_file, group_name= 'train_set', dset_name='image_data')
MS_bench = MicrostructureImageDataset(file_path=ms_file, group_name= 'benchmark_set', dset_name='image_data')

image_index = 1499
R = 73.658764

image = np.expand_dims(MS_bench[image_index].cpu().numpy(), axis=0)
train_feature = extract.full_computation(image) 
train_feature = torch.tensor(np.append(train_feature, [[1/R, R]], axis=1), device=device, dtype=torch.float32)



Initializing timer for this tag: rb loading and processing
rb loading and processing -> elapsed time: 0.1087
Initializing timer for this tag: full feature computation of 1 samples
fft -> elapsed time: 0.0049
xi computation -> elapsed time: 0.0104
band features -> elapsed time: 0.0233
projected edges -> elapsed time: 0.0002
local volume -> elapsed time: 0.0020
edge distributions -> elapsed time: 0.0155
full feature computation of 1 samples -> elapsed time: 0.0567
length of each feature type: [(1, 1), (1, 13), (1, 16), (1, 2), (1, 7), (1, 12)]


In [3]:
model_file = 'data/vrnn_therm2D_jit.pt'
model_jit = torch.jit.load('data/vrnn_therm2D_jit.pt')


kappa_pred = unpack_sym(model_jit(train_feature), dim=2)
print(kappa_pred.cpu().detach().numpy())

[[[0.17803453 0.01091091]
  [0.01091091 0.04025435]]]
